## Financial Analysis
### Steps:
#### 1] Web scraping to collect all the stocks fundamentals from Marketwatch
#### 2] Create a formatted dataframe which contains all the stocks fundamentals
#### 3] Pass the dataframe through eligibilty function to select best performing stocks  

#### Importing all the necessary libraries

In [6]:
import pandas as p
import numpy as np
import requests
import time
import datetime 
import pickle
import bs4 as bs
import yfinance as yfNew
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pyodbc

#### Read the list of symbols into list

In [7]:
#read symbols from file to python list
symbol = []
with open(r'C:\Users\SayaliGirish\Desktop\DWBI\StockList.csv') as f:
    for line in f:
        symbol.append(line.strip())
f.close

<function TextIOWrapper.close()>

In [14]:
print(len(symbol))

503


In [9]:
def getelementinlist(list,element):
    try:
        return list[element]
    except:
        return '-'

#### Web scraping from Marketwatch using Beautiful soup

In [10]:
def getfinancialreportingdf(ticker):

    # try:
    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'

    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml") #read in
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml") #read in


    # Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    interestexpenselist = []
    ebitdalist= []

    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])


    # Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

    # Variables        
    # eps = epslist[0]
    # epsgrowth = epslist[1]
    # netincome = netincomelist[0]
    # shareholderequity = equitylist[0]
    # roa = equitylist[1]

    # longtermdebt = longtermdebtlist[0]
    # interestexpense = interestexpenselist[0]
    # ebitda = ebitdalist[0]

    eps = getelementinlist(epslist,0)
    epsgrowth = getelementinlist(epslist,1)
    netincome = getelementinlist(netincomelist,0)
    shareholderequity = getelementinlist(equitylist,0)
    roa = getelementinlist(equitylist,1)

    longtermdebt = getelementinlist(longtermdebtlist,0)
    interestexpense =  getelementinlist(interestexpenselist,0)
    ebitda = getelementinlist(ebitdalist,0)

    ## Make it into Dataframes
    data= {'eps': eps,'epsgrowth': epsgrowth,'netincome': netincome,'shareholderequity': shareholderequity,'roa': 
                  roa,'longtermdebt': longtermdebt,'interestexpense': interestexpense,'ebitda': ebitda}
    df= pd.DataFrame(data,index=[2015,2016,2017,2018,2019])
    return df

#### Web scraping few other stocks fundamentals

In [11]:
def getFinancialProfiledf(ticker):

    # try:
    urlprofiles = 'https://www.marketwatch.com/investing/stock/'+ticker+'/profile'
    text_soup_profiles = BeautifulSoup(requests.get(urlprofiles).text,"lxml") #read in
    titlesprofiles = text_soup_profiles.findAll('p', {'class': 'column'})
    ROE=0
    Current_Ratio=0
    PE_Ratio=0
    debt_to_equity=0
    PB_Ratio=0
    for title in titlesprofiles:
        if 'Return on Equity' in title.text:
            ROE= [p.text for p in title.findNextSiblings(attrs={'class': 'data lastcolumn'}) if p.text]
        if 'Current Ratio' in title.text:
            Current_Ratio= [p.text for p in title.findNextSiblings(attrs={'class': 'data lastcolumn'}) if p.text]
        if 'P/E Current' in title.text:
            PE_Ratio= [p.text for p in title.findNextSiblings(attrs={'class': 'data lastcolumn'}) if p.text]
        if 'Total Debt to Total Equity' in title.text:
            debt_to_equity= [p.text for p in title.findNextSiblings(attrs={'class': 'data lastcolumn'}) if p.text]
        if 'Price to Book Ratio' in title.text:
            PB_Ratio= [p.text for p in title.findNextSiblings(attrs={'class': 'data lastcolumn'}) if p.text]    
    data1={'ROE':ROE,'Current_Ratio':Current_Ratio,'PE_Ratio':PE_Ratio,'debt_to_equity':debt_to_equity,'PB_Ratio':PB_Ratio}
    df=pd.DataFrame(data1,index=[2019])
    return df

#### Create a formatted dataframe

In [6]:
# Formatting df value to numerical
def format(list):
    newlist=[]
    number = 1
    for text in list:
        if text.endswith(')'):
            text = text[1:-1] # remove the parentheses
            number = -1
            
        if text.endswith('%'):
#             Then please make it into comma float
            endtext = float(text[:-1].replace(",",""))/100.0 * number 
        elif text.endswith('B'):
#             Then please times 1000000000
#             Change it into integer
            endtext = int(float(text[:-1].replace(",",""))*1000000000)* number 
        elif text.endswith('M'):
#             Then please times 1000000
#             Change it into integer
            endtext = int(float(text[:-1].replace(",",""))*1000000)* number
        elif text.endswith('T'):
#             Then please times 1000000000000
#             Change it into integer
            endtext = int(float(text[:-1].replace(",",""))*1000000000000)* number
        elif ',' in text:
#             Then please remove the ,
#             Then change it into int
            endtext = int(float(text.replace(",","")))* number
        elif text.endswith('-'):
#             Insert 0
            endtext = 0
        elif text.startswith(', '):
#             Insert 0
            endtext = 0
        else:
#           change to float
            endtext = float(text)* number 
        newlist.append(endtext)
    return newlist  

#### Create computted columns

In [7]:
# Getting financial reporting df
def getfinancialreportingdfformatted(ticker):
    df = getfinancialreportingdf(ticker)
    # Format all the number in dataframe
    dfformatted = df.apply(format)

    # Adding roe, interest coverage ratio
    dfformatted['roe'] = dfformatted.netincome/dfformatted.shareholderequity
    dfformatted['interestcoverageratio'] = dfformatted.ebitda/dfformatted.interestexpense

#     Insert ticker and df
    return dfformatted

####  Create a dataframe

In [16]:
i=1
Final_dataframe2 = []
while i < len(symbol):
    print(symbol[i])
    data2 =getFinancialProfiledf(symbol[i])
    data2['symbol'] = symbol[i] 
    Final_dataframe2.append(data2)
    i=i+1;
Final_dataframe2 = pd.concat(Final_dataframe2)

MMM
AOS
ABT
ABBV
ACN
ATVI
AYI
ADBE
AAP
AMD
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANDV
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BHGE
BLL
BAC
BAX
BBT
BDX
BRK.B
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BHF
BMY
AVGO
BF.B
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CBG
CBS
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DOV
DWDP
DPS
DTE
DUK
DRE
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
RE
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HRS
HIG
HAS
HCA
HCP
HP
HSIC
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HPQ
HUM
HBAN
HII
IDXX

#### Save the dataframe as csv file for further analysis

In [17]:
Final_dataframe2.to_csv('Stock_Statistics_latest.csv')

In [8]:
i=1
Final_dataframe = []
while i < len(symbol):
    print(symbol[i])
    data =getfinancialreportingdfformatted(symbol[i])
    data['Symbol'] = symbol[i] 
    Final_dataframe.append(data)
    i=i+1;
Final_dataframe = pd.concat(Final_dataframe)

MMM
AOS
ABT
ABBV
ACN
ATVI
AYI
ADBE
AAP
AMD
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANDV
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BHGE
BLL
BAC
BAX
BBT
BDX
BRK.B
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BHF
BMY
AVGO
BF.B
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CBG
CBS
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DOV
DWDP
DPS
DTE
DUK
DRE
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
RE
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HRS
HIG
HAS
HCA
HCP
HP
HSIC
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HPQ
HUM
HBAN
HII
IDXX

In [10]:
print(Final_dataframe)

       eps  epsgrowth   netincome  shareholderequity     roa  longtermdebt  \
2015  7.63     0.0000  5000000000        13110000000  0.4192    6790000000   
2016  7.73     0.0125  4840000000        11430000000  0.3476    8800000000   
2017  8.35     0.0809  5060000000        10300000000  0.3130   10720000000   
2018  8.13    -0.0263  4870000000        11560000000  0.3044   12160000000   
2019  9.09     0.1181  5360000000         9800000000  0.2684   13490000000   
...    ...        ...         ...                ...     ...           ...   
2015  1.16     0.0000   587000000         1310000000  0.1984    3640000000   
2016  0.68    -0.4138   339000000         1070000000  0.1350    4460000000   
2017  1.66     1.4412   819000000         1490000000  0.1944    4470000000   
2018  1.76     0.0602   862000000         1770000000  0.2061    4950000000   
2019  2.96     0.6796  1420000000         2190000000  0.2027    6440000000   

      interestexpense      ebitda       roe  interestcoveragera

In [9]:
Final_dataframe.to_csv('Stock_Statistics1.csv')

#### Function to check whether given stock is worth investing or not

In [1]:
def eligibilitycheck(ticker,dfformatted):
    
    legiblestock = True
    reasonlist=[]

    # print (dfformatted)
    # EPS increases over the year (consistent)
    for growth in dfformatted.epsgrowth:
        if growth<0:
            legiblestock = False
            reasonlist.append('there is negative growth '+str(growth))
            break
    # ROE > 0.15
    if dfformatted.roe.mean()<0.13:
            legiblestock = False
            reasonlist.append('roe mean is less than 0.13 '+ str(dfformatted.roe.mean()))
    # ROA > 0.07 (also consider debt to equity cause Assets = liabilities + equity)
    if dfformatted.roa.mean()<0.07:
            legiblestock = False
            reasonlist.append('roa mean is less than 0.07 ' + str(dfformatted.roa.mean()))
    # Long term debt < 5 * income
    if dfformatted.longtermdebt.tail(1).values[0]>5*dfformatted.netincome.tail(1).values[0]:
            legiblestock = False
            reasonlist.append('longtermdebt is 5 times the netincome ')
    # Interest Coverage Ratio > 3
    if dfformatted.interestcoverageratio.tail(1).values[0]<3:
            legiblestock = False
            reasonlist.append('Interestcoverageratio is less than 3 ')
    print (ticker,legiblestock,reasonlist)
    return reasonlist